In [1]:
import sys
sys.path.insert(1, '../')

In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import os
import yaml
from pydoc import locate
import tensorflow_datasets as tfds

In [3]:
def get_log_regression():
    return LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', C=1.0)

In [4]:
## define model id
model_id = '20200307-002920'
model_path = os.path.join('../logs', model_id, 'train', 'checkpoints')

In [5]:
def read_images(features):
    return features['image'], features['label']


def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

In [6]:
batch_size=128

In [7]:
test_dataset = tfds.load(name="cifar10", split="test")
test_dataset = test_dataset.map(read_images)
test_dataset = test_dataset.map(normalize)
test_dataset = test_dataset.repeat(1)
test_dataset = test_dataset.batch(batch_size, drop_remainder=False)

In [8]:
X_train = []
y_train = []
for x_batch, y_batch in test_dataset:
    X_train.extend(x_batch.numpy())
    y_train.extend(y_batch)

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [9]:
## train test split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.5, random_state=1)
print("Training data")
print(X_train.shape)
print(y_train.shape)
print("Testing data")
print(X_test.shape)
print(y_test.shape)

Training data
(5000, 32, 32, 3)
(5000,)
Testing data
(5000, 32, 32, 3)
(5000,)


In [10]:
print(np.min(X_train), np.max(X_train))
print(np.min(X_test), np.max(X_test))

0.0 1.0
0.0 1.0


In [11]:
print(y_train[:5], y_test[:5])
# [5 1 7 2 1] [7 9 3 8 0]

[5 9 4 5 7] [9 2 8 6 9]


## Test protocol #1 PCA features

In [12]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [13]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train.reshape((X_train.shape[0], -1)))

pca = PCA(n_components=128)

X_train_pca = pca.fit_transform(scaler.transform(X_train.reshape(X_train.shape[0], -1)))
X_test_pca = pca.transform(scaler.transform(X_test.reshape(X_test.shape[0], -1)))

print("PCA features")
print(X_train_pca.shape)
print(X_test_pca.shape)

PCA features
(5000, 128)
(5000, 128)


In [14]:
clf = get_log_regression()
clf.fit(X_train_pca, y_train)
print("PCA feature evaluation")
print("Train score:", clf.score(X_train_pca, y_train))
print("Test score:", clf.score(X_test_pca, y_test))

# PCA feature evaluation
# Train score: 0.4592
# Test score: 0.3632

PCA feature evaluation
Train score: 0.4482
Test score: 0.3692


In [15]:
os.path.join(model_path, "baseline_encoder.py")

'../logs/20200307-002920/train/checkpoints/baseline_encoder.py'

In [16]:
from models.cnn_small import SmallCNN


In [17]:
model = SmallCNN(64)
model(tf.ones((1,32,32,3)))
model.load_weights(os.path.join(model_path, 'model.h5'))

## Protocol #2 Linear separability evaluation

In [18]:
X_train_feature, _ = model(X_train)
print("Train features")
print(X_train_feature.shape)

Train features
(5000, 64)


In [19]:
X_test_feature , _ = model(X_test)
print("Test features")
print(X_test_feature.shape)

Test features
(5000, 64)


In [20]:
X_test_feature = X_test_feature.numpy()
X_train_feature = X_train_feature.numpy()

In [21]:
clf = get_log_regression()

scaler = preprocessing.StandardScaler()
scaler.fit(X_train_feature)

clf.fit(scaler.transform(X_train_feature), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
print("SimCLR feature evaluation")
print("Train score:", clf.score(scaler.transform(X_train_feature), y_train))
print("Test score:", clf.score(scaler.transform(X_test_feature), y_test))

# SimCLR feature evaluation
# Train score: 0.5946
# Test score: 0.5202

SimCLR feature evaluation
Train score: 0.5058
Test score: 0.464
